In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
import time

pd.options.mode.chained_assignment = None  # default='warn'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
## https://developers.refinitiv.com/article/find-your-right-companies-screener-eikon-data-apispython
# https://www.reddit.com/r/webdev/comments/t2s836/what_is_the_easiest_way_to_hide_your_api_key_on/
import eikon as ek
ek.set_app_key('341ab68907a740f38e7b07084d4055e3982b5bdf')

# M&A screening

<pre>•Deals zwischen Juni 2008 und September 2022
•Deal status: Completed & withdrawn
•Nation classification: US,Canada,Australia, United Kingdom
•Deal value: größer 1Mio USD
•M&A Type: Disclosed dollar value deal
•% of shares before deal kleiner 50%, danach größer 50%
•Company status: public</pre>

## retrieve sample

In [77]:
'''
unbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen
'''

firms = 'SCREEN(U(IN(DEALS)),' + ' \
            BETWEEN(TR.MnAAnnDate,20080107,20213112), \
            IN(TR.MnAAcquirorNation,"AU","GB","CA","US"), \
            IN(TR.MnATargetNation,"AU","GB","CA","US"), \
            IN(TR.MnAStatus,"W","SW","IW","DR","U","C"), \
            TR.MnADealValue(Scale=6)>=1, \
            IN(TR.MnADealType,"DI"), \
            TR.MnAPctOfSharesHeld6MonthsPrior<50, \
            TR.MnAPctOfSharesOwnedPostMerger>50, \
            IN(TR.MnAAcquirorPublicStatus,"P", "S"), \
            IN(TR.MnATargetPublicStatus,"P", "V", "S"), \
            CURN=USD) \
            '
            
fields = [
    'TR.MnASDCDealNumber,TR.MnAAnnDate,TR.MnADateUnconditional,TR.MnAEffectiveDate,TR.MnAStatus,TR.MnATarget,TR.MnATargetPermId,TR.MnATargetSDCCusip,TR.MnATargetNation,TR.MnATargetPublicStatus,TR.MnATargetTotalAssetsLTM,TR.MnATargetNetIncomeLTM,TR.MnATargetFinancialDate,TR.MnAAcquiror,TR.MnAAcquirorPermId,TR.MnAAcquirorSDCCusip,TR.MnAAcquirorNation,TR.MnAAcquirorPublicStatus,TR.MnAAcquirorTotalAssetsLTM,TR.MnAAcquirorNetIncomeLTM,TR.MnAAcquirorFinancialDate,TR.MnADealValue,TR.MnAPctOfSharesHeld6MonthsPrior,TR.MnAPctOfSharesOwnedPostMerger'
    ] #can be placed in just one string

params = {'Scale': 0, 'Curn': 'USD'} # set scale to 6 for mio values


df_unfiltered, err = ek.get_data(firms, fields, params)

df_unfiltered.head()
df_unfiltered.shape

'\nunbedingt "&" removen aus der excel funktion. den rest kann man anscheinend direkt übernehmen\n'

,Instrument,SDC Deal No,Date Announced,Date Effective or Unconditional,Date Effective,Deal Status,Target Full Name,Target PermID,Target 6-digit CUSIP,Target Nation,...,Acquiror PermID,Acquiror 6-digit CUSIP,Acquiror Nation,Acquiror Public Status,Acquiror Total Assets Last 12 Months,Acquiror Net Income Last 12 Months,Date of Acquiror Financials,Deal Value,Percentage of Shares Held by Acquiror 6 Months Prior to Announcement,Percentage of Shares Owned after Transaction
0,34391097466,2116013020,2009-10-07,2009-12-08,2009-12-08,Completed,Trinity Alps Resources Inc- Discovery Day Gold...,<NA>,25581K,United States,...,4298488815.0,029616,United States,Public,<NA>,<NA>,2009-10-07,3700000.0,0.0,75.0
1,34391276852,1761253020,2011-03-28,2011-05-02,2011-05-02,Completed,Fulton Street Brewery LLC,5035166382.0,14681L,United States,...,4296453080.0,035231,United States,Subsidiary,<NA>,<NA>,2011-03-28,22500000.0,0.0,58.0
2,34391279156,1780964020,2010-05-21,2010-06-29,2010-06-29,Completed,Michael Foods Inc,4295907158.0,594079,United States,...,5000878558.0,36224Z,United States,Subsidiary,<NA>,<NA>,2010-05-21,1700000000.0,0.0,100.0
3,34391281082,1794406020,2008-09-15,2008-10-30,2008-10-30,Completed,Napster Inc,4295899555.0,630797,United States,...,4295903556.0,086516,United States,Public,17522000000.0,1346000000.0,2008-08-30,126935000.0,0.0,90.191
4,34391290134,1879131020,2008-04-24,2008-09-29,2008-09-29,Completed,Wendy's International Inc,4295905330.0,950590,United States,...,4295905130.0,895927,United States,Public,1390735000.0,-59595000.0,2008-03-30,2345517000.0,0.0,100.0


(32330, 25)

In [18]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_Eikon_unfiltered.csv')

df_unfiltered.to_csv(path_with_time, sep=";", decimal=",", float_format='%.2f', index=False) #german decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0927_MnA_Sample_Eikon_unfiltered.csv saved


### prepare sample

In [79]:
df = df_unfiltered.copy()

# replace whitespaces in column names with underscore & make all column names lowercase
df.columns = df.columns.str.replace(' ', '_').str.lower()

# rename columns
df = df.rename(columns={"instrument": "eikon_deal_id", "total_assets_last_12_months": "target_total_assets_last_12_months", "date_of_target_financials": "target_date_of_financials", "date_of_acquiror_financials": "acquiror_date_of_financials"}) 

# convert date strings to date objects
for date_column in [col for col in df.columns if 'date' in col]:
    df[date_column] = pd.to_datetime(df[date_column]) 

# set permid from float to int
for permid_column in [col for col in df.columns if 'permid' in col]:
    df[permid_column] = df[permid_column].astype("Int64")

# filter df
condition = (
    (df["target_public_status"].isin(["Public", "Private", "Subsidiary"])) #exclude subsidiary
    & (df["acquiror_public_status"] == "Public") 
    & (~df['date_announced'].dt.year.isin([])) #exclude 2008, 2009, 2010
    & ((df["target_6-digit_cusip"].isna() == False) & (df["target_total_assets_last_12_months"].isna() == False)) #exclude empty PermID //some of them have financials -> keep them
    # & (df["Target 6-digit CUSIP"].isna() == True) # none missing
    # & (df['Total Assets Last 12 Months'] < df['Total Assets Last 12 Months'].quantile(0.1)) # < : exclude all above quantile
    )
df = df[condition]

# set order of public status column
df["target_public_status"] = pd.Categorical(df["target_public_status"], categories=["Public","Private","Subsidiary"], ordered=True)

# sort df by public/date/size
df = df.sort_values(by=["target_public_status", "date_announced", "deal_value"], ascending=[1,1,0]).reset_index(drop=True)

df.head()
df.shape

missing_isins = df[df["target_permid"].isna() == True].shape[0]
print(f"Obs with missing target permid: {missing_isins}")

,eikon_deal_id,sdc_deal_no,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_6-digit_cusip,target_nation,...,acquiror_permid,acquiror_6-digit_cusip,acquiror_nation,acquiror_public_status,acquiror_total_assets_last_12_months,acquiror_net_income_last_12_months,acquiror_date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391733382,1940967040,2008-01-07,2008-03-04,2008-03-04,Completed,Rider Resources Ltd,4295860890,76570J,Canada,...,4295861226,67072Q,Canada,Public,685633279.807306,26422119.63067,2007-12-31,282962189.054726,0.0,100.0
1,34391297274,1941059020,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,121368,United States,...,5048817617,85853F,Canada,Public,257300643.400273,24298108.793137,2008-03-31,76132000.0,0.0,100.0
2,34391733368,1940868040,2008-01-07,2008-03-05,2008-03-05,Completed,Bioscrypt Inc,4295860377,09065U,Canada,...,4295904213,50212A,United States,Public,1408352000.0,17666000.0,2007-12-31,43970500.298983,0.0,100.0
3,34391297298,1941224020,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,55272D,United States,...,4295914259,62845B,United States,Public,957343000.0,4226000.0,2007-12-31,57168000.0,0.0,100.0
4,34391733545,1941443040,2008-01-08,2008-04-15,2008-04-15,Completed,Metco Resources Inc,4295921364,591467,Canada,...,4295861480,106902,Canada,Public,591907868.325974,23491569.650743,2007-12-31,11335187.760779,0.0,100.0


(4748, 25)

Obs with missing target permid: 121


## cusip to ISIN (totally unreliable, use permid to isin instead)

In [ ]:
firms_cusips = df["target_6-digit_cusip"].tolist()

df_cusips = ek.get_symbology(firms_cusips, from_symbol_type='CUSIP', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_cusips = df_cusips[df_cusips["ISIN"].isna() == False]
df_cusips

## permid to isin

#### target

In [80]:
# create target_permid list
target_cusips = df["target_permid"].astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df_isin_target_retrieve, err = ek.get_data(target_cusips, fields, params)

df_isin_target_retrieve

,Instrument,ISIN,CUSIP,RIC
0,4295860890,CA7656982047,765698204,RRZ.TO^C08
1,4296470998,US1213681042,121368104,BPAB.PK^E08
2,4295860377,CA09065U1066,09065U106,BYT.TO^C08
3,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
4,4295921364,CA5914671055,591467105,MKO.V^D08
...,...,...,...,...
4743,4296105218,,,
4744,5037841945,,,
4745,4295908112,US8782371061,878237106,TECD.OQ^G20
4746,4295903356,US8873173038,887317303,TWX.N^F18


In [81]:
df_isin_target = df_isin_target_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_target[columnname] = df_isin_target[columnname].replace({"":np.nan})

df_isin_target = df_isin_target.rename(columns={"Instrument": "target_permid"})

# add prefix to columns (except first column)
new_names = [(i,'Target_'+i) for i in df_isin_target.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "Target_old"}
df_isin_target.rename(columns = dict(new_names), inplace=True)
# convert column names to lowercase
df_isin_target.columns = df_isin_target.columns.str.lower()

# only display rows in which isin is not missing
df_isin_target = df_isin_target[df_isin_target["target_isin"].isna() == False]

df_isin_target

,target_permid,target_isin,target_cusip,target_ric
0,4295860890,CA7656982047,765698204,RRZ.TO^C08
1,4296470998,US1213681042,121368104,BPAB.PK^E08
2,4295860377,CA09065U1066,09065U106,BYT.TO^C08
3,4295907031,US55272D1063,55272D106,MFBC.OQ^G08
4,4295921364,CA5914671055,591467105,MKO.V^D08
...,...,...,...,...
4737,4295858262,AU000000MOY9,<NA>,MOY.AX^E20
4738,4296534968,US28849P1003,28849P100,ELLI.N^D19
4739,5001225083,US76132B1061,76132B106,SALE.O^E17
4745,4295908112,US8782371061,878237106,TECD.OQ^G20


#### acq

In [69]:
acquiror_cusips = df["acquiror_permid"].astype(str).tolist() #first convert to int (without decimal), then to str, since get_data needs strings in the list
fields = ['TR.ISIN', 'TR.CUSIP', 'TR.RIC'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df_isin_acquiror_retrieve, err = ek.get_data(acquiror_cusips, fields, params)


df_isin_acquiror = df_isin_acquiror_retrieve.copy()
#replace empty strings with np nan
for columnname in ['ISIN', 'CUSIP', 'RIC']:
  df_isin_acquiror[columnname] = df_isin_acquiror[columnname].replace({"":np.nan})

df_isin_acquiror = df_isin_acquiror.rename(columns={"Instrument": "acquiror_permid"})

# add prefix to columns (except first column)
new_names = [(i,'Acquiror_'+i) for i in df_isin_acquiror.iloc[:, 1:].columns.values] #set up dict entries for each columns name {"old": "acquiror_old"}
df_isin_acquiror.rename(columns = dict(new_names), inplace=True)

# convert column names to lowercase
df_isin_acquiror.columns = df_isin_acquiror.columns.str.lower()

# only display rows in which isin is not missing
df_isin_acquiror = df_isin_acquiror[df_isin_acquiror["acquiror_isin"].isna() == False]
df_isin_acquiror

,acquiror_permid,acquiror_isin,acquiror_cusip,acquiror_ric
0,4295861226,CA67072Q1046,67072Q104,NVA.TO
1,5048817617,CA85853F1053,85853F105,SJ.TO
2,4295904213,US4519061018,451906101,IDNX.OQ^H06
3,4295914259,US62845B1044,62845B104,MFSF.OQ^D20
4,4295861480,CA1069027037,106902703,BWR.TO^I11
...,...,...,...,...
4743,4295913094,US8936411003,893641100,TDG.N
4744,4295907298,US5503511009,550351100,LUNA.OQ
4745,4295899452,US87162W1009,87162W100,SNX.N
4746,4295907310,US25470F1049,25470F104,DISCA.OQ


#### merge acq / target ISINs on PermID (df also includes obs with missing isin)

In [72]:
df_isin_acquiror_merge = df_isin_acquiror.copy()
df_isin_target_merge = df_isin_target.copy()

# perm IDs are stored as string -> convert to Int64
for columnname in ["acquiror_permid", "target_permid"]:
    df[columnname] = df[columnname].astype("Int64")

df_isin_acquiror_merge["acquiror_permid"] = df_isin_acquiror_merge["acquiror_permid"].astype("Int64")
df_isin_target_merge["target_permid"] = df_isin_target_merge["target_permid"].astype("Int64")

# merge acq / target ISINs on PermID
df_final = df.merge(df_isin_acquiror_merge.iloc[:,:2], how="left", on="acquiror_permid")
df_final = df_final.merge(df_isin_target_merge.iloc[:,:2], how="left", on="target_permid")

# there are duplicates since multiple acquiror possible
df_final = (df_final.drop_duplicates()
            .reset_index(drop=True))

# create identifier based on acq + target isin
df_final["acq_target_isin"] = df_final["acquiror_isin"] + df_final["target_isin"]

# shift columns (based on dict)
columns_to_move = {"acquiror_isin": "acquiror_permid", "target_isin": "target_permid", "acq_target_isin": "sdc_deal_no"}
for key, value in columns_to_move.items():
    poped_col = df_final.pop(key)
    df_final.insert(df_final.columns.get_loc(value)+1, key, poped_col)

# replace whitespaces in column names with underscore & make all column names lowercase
df_final.columns = df_final.columns.str.replace(' ', '_').str.lower()

# remove obs with missing target ISIN
# df_final = df_final[df_final["target_isin"].isna() == False] # ignore them. some of them have a DE isin based on the given cusip, which doesnt make sense

df_final.head()
df_final.shape

missing_isins = df_final[df_final["target_isin"].isna() == True].shape[0]
print(f"Obs with missing target ISIN: {missing_isins}")

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_isin,...,acquiror_isin,acquiror_6-digit_cusip,acquiror_nation,acquiror_public_status,acquiror_total_assets_last_12_months,acquiror_net_income_last_12_months,acquiror_date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391733382,1940967040,CA67072Q1046CA7656982047,2008-01-07,2008-03-04,2008-03-04,Completed,Rider Resources Ltd,4295860890,CA7656982047,...,CA67072Q1046,67072Q,Canada,Public,685633279.807306,26422119.63067,2007-12-31,282962189.054726,0.0,100.0
1,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,US1213681042,...,CA85853F1053,85853F,Canada,Public,257300643.400273,24298108.793137,2008-03-31,76132000.0,0.0,100.0
2,34391733368,1940868040,US4519061018CA09065U1066,2008-01-07,2008-03-05,2008-03-05,Completed,Bioscrypt Inc,4295860377,CA09065U1066,...,US4519061018,50212A,United States,Public,1408352000.0,17666000.0,2007-12-31,43970500.298983,0.0,100.0
3,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,US55272D1063,...,US62845B1044,62845B,United States,Public,957343000.0,4226000.0,2007-12-31,57168000.0,0.0,100.0
4,34391733545,1941443040,CA1069027037CA5914671055,2008-01-08,2008-04-15,2008-04-15,Completed,Metco Resources Inc,4295921364,CA5914671055,...,CA1069027037,106902,Canada,Public,591907868.325974,23491569.650743,2007-12-31,11335187.760779,0.0,100.0


(4748, 28)

Obs with missing target ISIN: 2014


### firm SIC

In [84]:
df_trbc = df_final.copy()

df_acquiror_trbc, df_target_trbc = pd.DataFrame(), pd.DataFrame()
target_acq_isins = {"acquiror_isin": df_acquiror_trbc, "target_isin": df_target_trbc}

# retrieve acq and target trbc codes
for isin_column, datafr in target_acq_isins.items():
    firms = df_final[isin_column].astype(str).tolist()
    fields = ['TR.TRBCIndustryGroupCode', 'TR.TRBCIndustryGroup'] # check the data item browser
    params = {'Scale': 2, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

    datafr, err = ek.get_data(firms, fields, params)

    # replace whitespaces in column names with underscore & make all column names lowercase
    datafr.columns = datafr.columns.str.replace(' ', '_').str.lower()

    datafr = datafr.rename(columns={"instrument": isin_column})
    datafr.head()
    datafr.shape

    # merge with df_trbc
    df_trbc = df_trbc.merge(datafr, how="left", on=isin_column)
    df_trbc = df_trbc.drop_duplicates()

    # shift columns (based on dict)
    columns_to_move = {list(df_trbc.columns)[-1]: isin_column, list(df_trbc.columns)[-2]: isin_column}
    for key, value in columns_to_move.items():
        poped_col = df_trbc.pop(key)
        df_trbc.insert(df_trbc.columns.get_loc(value)+3, key, poped_col)


df_trbc = df_trbc.rename(columns={"trbc_industry_group_code_x": "acquiror_trbc_industry_group_code", "trbc_industry_group_name_x": "acquiror_trbc_industry_group_name", "trbc_industry_group_code_y": "target_trbc_industry_group_code", "trbc_industry_group_name_y": "target_trbc_industry_group_name"})

# mark 551020 Investment Banking & Investment Services (n=114)
df_trbc["investment acquiror"] = np.where((df_trbc["acquiror_trbc_industry_group_code"].isna() == False) & (df_trbc["acquiror_trbc_industry_group_code"] == 551020), 1, 0)
poped_col = df_trbc.pop("investment acquiror")
df_trbc.insert(df_trbc.columns.get_loc("acquiror_trbc_industry_group_name")+1, "investment acquiror", poped_col)

df_trbc.head()
df_trbc.shape

,acquiror_isin,trbc_industry_group_code,trbc_industry_group_name
0,CA67072Q1046,501020,Oil & Gas
1,CA85853F1053,513010,Paper & Forest Products
2,US4519061018,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships"
3,US62845B1044,551010,Banking Services
4,CA1069027037,512010,Metals & Mining


(4748, 3)

,target_isin,trbc_industry_group_code,trbc_industry_group_name
0,CA7656982047,501020,Oil & Gas
1,US1213681042,513010,Paper & Forest Products
2,CA09065U1066,572010,Software & IT Services
3,US55272D1063,551010,Banking Services
4,CA5914671055,512010,Metals & Mining


(4748, 3)

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_permid,target_isin,...,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,investment acquiror,acquiror_public_status,acquiror_total_assets_last_12_months,acquiror_net_income_last_12_months,acquiror_date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391733382,1940967040,CA67072Q1046CA7656982047,2008-01-07,2008-03-04,2008-03-04,Completed,Rider Resources Ltd,4295860890,CA7656982047,...,501020,Oil & Gas,0,Public,685633279.807306,26422119.63067,2007-12-31,282962189.054726,0.0,100.0
1,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,4296470998,US1213681042,...,513010,Paper & Forest Products,0,Public,257300643.400273,24298108.793137,2008-03-31,76132000.0,0.0,100.0
2,34391733368,1940868040,US4519061018CA09065U1066,2008-01-07,2008-03-05,2008-03-05,Completed,Bioscrypt Inc,4295860377,CA09065U1066,...,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships",0,Public,1408352000.0,17666000.0,2007-12-31,43970500.298983,0.0,100.0
3,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",4295907031,US55272D1063,...,551010,Banking Services,0,Public,957343000.0,4226000.0,2007-12-31,57168000.0,0.0,100.0
4,34391733545,1941443040,CA1069027037CA5914671055,2008-01-08,2008-04-15,2008-04-15,Completed,Metco Resources Inc,4295921364,CA5914671055,...,512010,Metals & Mining,0,Public,591907868.325974,23491569.650743,2007-12-31,11335187.760779,0.0,100.0


(4748, 33)

In [75]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_EikonWithPrivateAndTRBC_4748.csv')

df_trbc.to_csv(path_with_time, sep=";", decimal=",", float_format='%.3f', index=False) #english decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0927_MnA_Sample_EikonWithPrivateAndTRBC_4848.csv saved


### clean firm names

In [105]:
df_name = df_trbc.copy()

for old_name_column, new_name_column in {"target_full_name": "target_full_name_cleaned", "acquiror_full_name": "acquiror_full_name_cleaned"}.items():
    df_name[new_name_column] = df_name[old_name_column]
    poped_col = df_name.pop(new_name_column)
    df_name.insert(df_name.columns.get_loc(old_name_column)+1, new_name_column, poped_col)

    # remove locations, which are listed after a comma
    df_name[new_name_column] = df_name[new_name_column].str.split(',').str[0]

    df_name[new_name_column] = df_name[new_name_column].str.replace("&", "and")
    df_name[new_name_column] = df_name[new_name_column].str.replace("and Co", "and Co.")
    # remove legal addons in firm names
    df_name[new_name_column] = df_name[new_name_column].str.replace(",| Inc| Corp| Limited| Ltd| SA| AG| Co$| plc| LP| LLC| LLC| Corporation| Holdings|\(Holdings\)PLC| Co Ltd| NL$", "", regex=True, case=False) #. is used a a placeholder. -> put it in character class by using []

df_name.head()
df_name.shape

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_full_name_cleaned,target_permid,...,acquiror_trbc_industry_group_code,acquiror_trbc_industry_group_name,investment acquiror,acquiror_public_status,acquiror_total_assets_last_12_months,acquiror_net_income_last_12_months,acquiror_date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction
0,34391733382,1940967040,CA67072Q1046CA7656982047,2008-01-07,2008-03-04,2008-03-04,Completed,Rider Resources Ltd,Rider Resources,4295860890,...,501020,Oil & Gas,0,Public,685633279.807306,26422119.63067,2007-12-31,282962189.054726,0.0,100.0
1,34391297274,1941059020,CA85853F1053US1213681042,2008-01-07,2008-04-01,2008-04-01,Completed,Burke-Parsons-Bowlby Corp,Burke-Parsons-Bowlby,4296470998,...,513010,Paper & Forest Products,0,Public,257300643.400273,24298108.793137,2008-03-31,76132000.0,0.0,100.0
2,34391733368,1940868040,US4519061018CA09065U1066,2008-01-07,2008-03-05,2008-03-05,Completed,Bioscrypt Inc,Bioscrypt,4295860377,...,521020,"Machinery, Tools, Heavy Vehicles, Trains & Ships",0,Public,1408352000.0,17666000.0,2007-12-31,43970500.298983,0.0,100.0
3,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",MFB,4295907031,...,551010,Banking Services,0,Public,957343000.0,4226000.0,2007-12-31,57168000.0,0.0,100.0
4,34391733545,1941443040,CA1069027037CA5914671055,2008-01-08,2008-04-15,2008-04-15,Completed,Metco Resources Inc,Metco Resources,4295921364,...,512010,Metals & Mining,0,Public,591907868.325974,23491569.650743,2007-12-31,11335187.760779,0.0,100.0


(4748, 35)

### high quality filter

In [112]:
conditions = (
(df_name["acq_target_isin"].isna() == False) &
(df_name["target_trbc_industry_group_code"].isna() == False) &
(df_name["acquiror_trbc_industry_group_code"].isna() == False) &
(df_name["target_net_income_last_12_months"].isna() == False) &
(df_name["investment acquiror"] == 0)
)

df_name["high_quality_observation"] = np.where(conditions == True, 1, 0)

df_name["high_quality_observation"].value_counts()

df_name = df_name.sort_values(by=["high_quality_observation", "target_public_status", "date_announced", "deal_value"], ascending=[0,1,1,0]).reset_index(drop=True)


df_name.head()
df_name.shape

1    2407
0    2341
Name: high_quality_observation, dtype: int64

,eikon_deal_id,sdc_deal_no,acq_target_isin,date_announced,date_effective_or_unconditional,date_effective,deal_status,target_full_name,target_full_name_cleaned,target_permid,...,acquiror_trbc_industry_group_name,investment acquiror,acquiror_public_status,acquiror_total_assets_last_12_months,acquiror_net_income_last_12_months,acquiror_date_of_financials,deal_value,percentage_of_shares_held_by_acquiror_6_months_prior_to_announcement,percentage_of_shares_owned_after_transaction,high_quality_observation
0,34391733382,1940967040,CA67072Q1046CA7656982047,2008-01-07,2008-03-04,2008-03-04,Completed,Rider Resources Ltd,Rider Resources,4295860890,...,Oil & Gas,0,Public,685633279.807306,26422119.63067,2007-12-31,282962189.054726,0.0,100.0,1
1,34391733368,1940868040,US4519061018CA09065U1066,2008-01-07,2008-03-05,2008-03-05,Completed,Bioscrypt Inc,Bioscrypt,4295860377,...,"Machinery, Tools, Heavy Vehicles, Trains & Ships",0,Public,1408352000.0,17666000.0,2007-12-31,43970500.298983,0.0,100.0,1
2,34391297298,1941224020,US62845B1044US55272D1063,2008-01-08,2008-07-18,2008-07-18,Completed,"MFB Corp,Mishawaka,Indiana",MFB,4295907031,...,Banking Services,0,Public,957343000.0,4226000.0,2007-12-31,57168000.0,0.0,100.0,1
3,34391297367,1941680020,US9841022026US82511E1091,2008-01-09,2008-05-30,2008-05-30,Completed,"Shore Financial Corp,Onley, Virginia",Shore Financial,4295907912,...,Banking Services,0,Public,561170000.0,6811000.0,2007-12-31,56481000.0,0.0,100.0,1
4,34391297484,1942305020,US2922181043US02342J1016,2008-01-10,2008-10-31,2008-10-31,Completed,AmCOMP Inc,AmCOMP,4295910045,...,Insurance,0,Public,3081413000.0,114504000.0,2008-06-30,190579000.0,0.0,100.0,1


(4748, 36)

In [116]:
## save file with dynamic file name
dir = Path("W:/019_Glassdoor/1 Data/1 Glassdoor Links/MnA Sample Eikon")
path_with_time = Path.joinpath(dir, f'{time.strftime("%m%d")}_MnA_Sample_EikonWithPrivate_CleanNames_4748.csv')

df_name.to_csv(path_with_time, sep=";", decimal=",", float_format='%.3f', index=False) #english decimal

print(f"{path_with_time} saved")

W:\019_Glassdoor\1 Data\1 Glassdoor Links\MnA Sample Eikon\0927_MnA_Sample_EikonWithPrivate_CleanNames_4748.csv saved


# examples

## get panel data

### firm fundamentals

In [ ]:
firms = ['5030853586', '4295906579'] # check the screener
fields = ['TR.Revenue.date','TR.Revenue','TR.GrossProfit', 'TR.ISIN', 'TR.RIC', 'TR.CommonName'] # check the data item browser
params = {'Scale': 2, 'SDate': -15, 'EDate': -10, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, fields, params)

df

### symbology converter

In [ ]:
firms = ['GOOG.O', 'MSFT.O', 'FB.O', 'AMZN.O', 'TWTR.K'] # check the screener
fields = ['TR.RIC', 'TR.CUSIP', 'TR.SEDOL', 'TR.ISIN', 'TR.OrganizationID'] # check the data item browser
params = {'Scale': 6, 'SDate': 0, 'EDate': 0, 'FRQ': 'FY', 'Curn': 'USD'} # set EDate to minus values for the most recent years

df, err = ek.get_data(firms, fields, params)

df

In [ ]:
firm_ids = df["RIC"].tolist()
df_symb = ek.get_symbology(firm_ids, from_symbol_type='RIC', to_symbol_type=None, raw_output=False, debug=False)
# to_symbol_type=None -> all are requested. CUSIP , ISIN , SEDOL , RIC , ticker , lipperID , IMO , and OAPermID

df_symb